In [15]:
_RIOT_API_KEY='RGAPI-79bd2f52-e028-4e9b-9a04-1764b72aecd8'
from riotwatcher import LolWatcher
import helper_functions as help
from datetime import datetime
lol_watcher = LolWatcher(_RIOT_API_KEY)


1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

In [43]:
match_list=help.get_matchlist_by_summoner_name('souporsecret', lol_watcher)
len(match_list)
arena_match = lol_watcher.match.by_id('na1', match_list[0])
flex_match = lol_watcher.match.by_id('na1', match_list[4])


In [195]:
info = flex_match['info']
column_set=set(info['participants'][0].keys())
column_set.remove('perks')
column_set.remove('challenges')
column_list = list(column_set)
column_list

['lane',
 'getBackPings',
 'goldEarned',
 'sightWardsBoughtInGame',
 'totalTimeSpentDead',
 'largestKillingSpree',
 'spell4Casts',
 'kills',
 'trueDamageTaken',
 'bountyLevel',
 'damageDealtToObjectives',
 'summoner1Casts',
 'totalAllyJungleMinionsKilled',
 'wardsKilled',
 'magicDamageDealtToChampions',
 'trueDamageDealtToChampions',
 'dragonKills',
 'assistMePings',
 'spell1Casts',
 'totalEnemyJungleMinionsKilled',
 'pushPings',
 'teamId',
 'inhibitorTakedowns',
 'unrealKills',
 'puuid',
 'totalMinionsKilled',
 'win',
 'consumablesPurchased',
 'needVisionPings',
 'quadraKills',
 'inhibitorsLost',
 'tripleKills',
 'damageSelfMitigated',
 'totalUnitsHealed',
 'deaths',
 'totalDamageShieldedOnTeammates',
 'championTransform',
 'damageDealtToTurrets',
 'riotIdName',
 'nexusTakedowns',
 'visionScore',
 'enemyMissingPings',
 'item1',
 'spell3Casts',
 'spell2Casts',
 'dangerPings',
 'physicalDamageDealt',
 'nexusKills',
 'wardsPlaced',
 'firstTowerKill',
 'baitPings',
 'playerAugment3',
 'ob

In [198]:
participants = []
for row in info['participants']:
    row.remove('challenges')
    print(row)
    #participants_row = {}

AttributeError: 'dict' object has no attribute 'remove'

In [95]:

match_time = datetime.fromtimestamp(int(flex_match['info']['gameCreation']/1000)).strftime('%Y-%m-%d %H:%M:%S')
print('match time: ', match_time)
print('match duration:', flex_match['info']['gameDuration'], 'seconds' )

match time:  2023-07-17 20:53:05
match duration: 2220 seconds


'2023-07-17 20:53:05'